
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Exercise Session 6 - Odometry and Localisation<br/> Duration : 4 hours (2 in session + 2 at home)</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">How to use this notebook?</span><br/>
    This notebook is made of text cells and code cells. The code cells have to be <strong>executed</strong> to see the result of the program. To execute a cell, simply select it and click on the "play" button (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) in the tool bar just above the notebook, or type <code>shift + enter</code>. It is important to execute the code cells in their order of appearance in the notebook.<br/>
You can make use of the table of contents to navigate easily between sections.
</div>

<br/>

<div style="justify;text-justify: inter-word">
So that you may familiarise with the notebooks and the basic python syntax, the exercises are provided in notebook form and whenever there are any calculations to be made, we encourage you to do them by code. Also, if you want to take notes, we encourage you to use the markdown or Raw NBConvert cells. 
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Learning-Goals" data-toc-modified-id="Learning-Goals-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learning Goals</a></span></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Odometry" data-toc-modified-id="Odometry-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Odometry</a></span><ul class="toc-item"><li><span><a href="#Introduction-and-goal-of-odometry-exercise" data-toc-modified-id="Introduction-and-goal-of-odometry-exercise-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Introduction and goal of odometry exercise</a></span></li><li><span><a href="#Move-and-turn" data-toc-modified-id="Move-and-turn-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Move and turn</a></span></li></ul></li><li><span><a href="#Localisation" data-toc-modified-id="Localisation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Localisation</a></span><ul class="toc-item"><li><span><a href="#Computing-the-probabilities" data-toc-modified-id="Computing-the-probabilities-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Computing the probabilities</a></span></li><li><span><a href="#Implementation-on-Thymio" data-toc-modified-id="Implementation-on-Thymio-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Implementation on Thymio</a></span></li></ul></li></ul></div>

# Learning Goals


- Program Thymio to observe odometry errors. 


- Compute a probabilistic localisation with sensing and motion uncertainties. 


- Program a simple localisation algorithm that takes into account sensing and motion uncertainties.


# Requirements

- Thymio

- The A3 sheet with the stripe pattern (localization-map.pdf)

![localisation map](images/localization-map.png)


In [ ]:
!pip install --upgrade tdmclient

# Odometry

## Introduction and goal of odometry exercise

Odometry is an essential component of robot localisation, but is subject to several problems, linking theory and practice. The goal of this exercise is to observe several of these problems by implementinga very simple trajectory using the Thymio robot.

## Move and turn


Implement on Thymio a program that makes it continuously turn and move forward:


    1. Turn 360 + 180 degrees


    2. Move forward a distance corresponding to the movement the wheel did during the turning step.


Of course this movement needs to be carefully calibrated for your specific robot. Therefore first plan and implement a calibration procedure.


Place a pencil in the central hole of Thymio and draw the trajectory. Please note that this will require two A4 sheets of paper. 


Observe the result, iterate to improve the result (do you need other calibrations?) and comment the various errors you correct.


<blockquote>
    
<span style="color: #2980B9 ;">


To calibrate the odometry in rotation (the only needed here) based on time and speed (the only possibility here, having no encoders) it is useful to have a program that allows to set a speed and count time. The following program allows to start and stop rotating, enabling to count how much time (in 10ms units) is taken to make 10 revolutions. 


    var counting = 0
    var time = 0

    timer.period[0] = 10 # milliseconds

    onevent button.right # start counting
      counting = 1
      time = 0
      motor.left.target = 100
      motor.right.target = -100

    onevent button.center # stop counting
      counting = 0
      motor.left.target = 0
      motor.right.target = 0

    onevent timer0
      if counting == 1 then
        time = time + 1
      end


</span> 
</blockquote>

<span style="color: #2980B9 ;">
    
Using Python (you should still use Aseba studio to observe the variable time):

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [ ]:
%%run_python

counting = 0
time = 0

timer_period[0] = 10 # milliseconds

@onevent 
def button_right(): # start counting
    global counting, time, motor_left_target, motor_right_target
    counting = 1
    time = 0
    motor_left_target = 100
    motor_right_target = -100

@onevent 
def button_center(): # stop counting
    global counting, motor_left_target, motor_right_target
    counting = 0
    motor_left_target = 0
    motor_right_target = 0

@onevent 
def timer0():
    global counting, time
    if counting == 1:
        time = time + 1

<blockquote>
    
<span style="color: #2980B9 ;">


Once you know the time required to make one revolution of the robot (in this case on itself at the given speed 100,-100), you need to write a code allowing to move and turn in repetition:


    const FORWARD = 1
    const TURN = 2
    const ONETURN = 8873
    const SAMPLING = 5

    var state = 0
    var time = 0

    timer.period[0] = SAMPLING # 5 ms resolution

    onevent button.forward
      state = FORWARD
      time = 0
      motor.left.target = 100
      motor.right.target = 100

    onevent button.center
      state = 0
      time = 0
      motor.left.target = 0
      motor.right.target = 0


    onevent timer0
      time = time + 1
      if state == FORWARD then
        if time > ONETURN*3/SAMPLING/2 then
          time = 0
          state = TURN
          motor.left.target = 100
          motor.right.target = -100
        end
      end
      if state == TURN then
        if time > ONETURN*3/SAMPLING/2 then
          time = 0
          state = FORWARD
          motor.left.target = 100
          motor.right.target = 99
        end
      end


The figure below shows an example of several runs. Several observations can be done:


![Problems](Images/variousproblems.jpg)
*Example with bad calibration and non-straight motion.*


+ The robot does not go straight, this is a problem of calibration of the two speeds. This has been corrected in the program above by giving different speeds to the two wheels.

+ In turn 5 the robot slipped, turning less than required.

+ The calibration for rotation seems correct. If the calibration is not correct and the robot goes staight, the global shape is very similar.



The Figure below shows a much better run. In this case both the turning calibration and the straight motion are good, and we see on the sides that the dispersion of the turning points is much larger vertically than horizontally, showing the effect of the errors of odometry seen in the course.



![Noise Only](Images/noiseonly.jpg)
*Example with good calibration and straight motion.*

</span> 
</blockquote>

<span style="color: #2980B9 ;">
Using Python:

In [ ]:
%%run_python

FORWARD = 1
TURN = 2
ONETURN = 8873
SAMPLING = 5

state = 0
time = 0

timer_period[0] = SAMPLING # 5 ms resolution

@onevent 
def button_forward():
    global state, FORWARD, time, motor_left_target, motor_right_target
    state = FORWARD
    time = 0
    motor_left_target = 100
    motor_right_target = 100

@onevent 
def button_center():
    global state, time, motor_left_target, motor_right_target
    state = 0
    time = 0
    motor_left_target = 0
    motor_right_target = 0

@onevent 
def timer0():
    global state, FORWARD, TURN, ONETURN, SAMPLING, time, motor_left_target, motor_right_target
    time = time + 1
    if state == FORWARD:
        if time > ONETURN*3//SAMPLING//2:
            time = 0
            state = TURN
            motor_left_target = 100
            motor_right_target = -100
            
    if state == TURN:
        if time > ONETURN*3//SAMPLING//2:
            time = 0
            state = FORWARD
            motor_left_target = 100
            motor_right_target = 99


# Localisation

The goal of this second part is to practice the localisation as explained during the course.

## Computing the probabilities


If you do not feel confident with the explanation given during the course, do again all computations, for instance in a spreadsheet.


Consider the map below with the initial position of the robot, an initial probability distribution, and its movement restricted to moving from left to right. In the motion model, there is an error of motion with a probability of $10\%$ of incorrect motion to the neighbour case, and therefore $80\%$ or correct displacement. For the perception model, consider a probability of wrong measurement of $0.25$, meaning that if the ground is dark there is a probability of $25\%$ of reading a light color and viceversa.


Compute using a Bayes filter (also called Markov localisation):


- The prediction probabilities and measurement update based on the given movement (one step), assuming a correct movement and measurement. Give the final probability distribution after this step.


Complete as many rows as needed, indicating concisely what is performed on the left.

![Localisation](Images/localization.png)


| Map                        |       | X     |       |       | X     | X       |       |       |       |      |
|----------------------------|-------|-------|-------|-------|-------|---------|-------|-------|-------|------|
| Pos                        |  0    |  0    | 0     |  0    |  0    |   r     |  0    |  0    |  0    |  0   |
| Initial state.             |  0    |  0    | 0     |  0    |  0    |   1     |  0    |  0    |  0    |  0   |
| After move                 |  0    |  0    | 0     |  0    |  0    |  0.1    | 0.8   | 0.1   |  0    |  0   |
| Meas. prob. (no wall)      |  0.75 |  0.25 | 0.75  | 0.75  | 0.25  |  0.25   | 0.75  | 0.75  | 0.75  | 0.75 |
| Prod.                      |  0    |  0    | 0     |  0    |  0    |  0.025  | 0.6   | 0.075 |  0    |  0   |
| Normalized (/0.7)  |  0    |  0    | 0     |  0    |  0    | 0.0357 | 0.857 |0.107|  0    |  0    |

## Implementation on Thymio

Implement the localisation algorithm on the Thymio robot. You can simplify the program by moving yourself the robot from one position to the other and pressing a button when you are ready to make the measurement, simplifying the motion control of the robot and focusing on the computation of the probability. Use the same example as given in the course, based on a set of grey levels on the ground. The corresponding A3 ground is provided. 

    
<span style="color: #2980B9 ;">


>  The **Aseba code** is provided in the folder AESL solutions 
    
![Map](Images/map-robot.png)

<span style="color: #2980B9 ;">
    
    
> This first Python code is providing a simplified correction. You will need to move the robot by yourself. 

So for example put the robot on the first case 0, press the `button_center` that will show the initial belief (which is uniform) on the `leds_circle`. Then, press the `button_forward` that will call the function sense() and show the result beliefs. As you are on a black case, the robot indicate likely positions: 0, 1, 4, 5 ,6. Then you should move the robot to the case 1 and press again the `button_forward`, to show the result beliefs after the move() function (beliefs are basically shifted to the right). And you can press again the `button_forward` to show the beliefs after a new step of sensing, thanks to the function sense(). Now, the robot show strong beliefs for the positions 1, 5 and 6, as there are positions where we can find two black cases in a row. You can repeat these actions and see the result beliefs of the localisation algorithm.

In [ ]:
%%run_python

# Constants
LED = 32            # scale factor for displaying beliefs in leds
THRESHOLD = 400     # value below black is detected
P_HIT = 9           # probability of detecting black when on black
                    #              or white when on white
P_MISS = 1          # probability of detecting black when on white
                    #              or white when on black

# The world: 2 black, 2 white, 3 black, 1 white
world  = [1, 1, 0, 0, 1, 1, 1, 0]

# Beliefs about the localisation
beliefs = [0, 0, 0, 0, 0, 0, 0, 0]

# state  0 = sense, 1 = move
# hit    1 = on black, 0 = on white
# temp   Variable for cyclic move
# sum_b  Sum of beliefs for normalization

# Display the beliefs in the circle leds
def display_beliefs():
    global beliefs, LED, leds_circle
    leds_circle = [beliefs[0]//LED, beliefs[1]//LED, beliefs[2]//LED, beliefs[3]//LED,
                   beliefs[4]//LED, beliefs[5]//LED, beliefs[6]//LED, beliefs[7]//LED]
        
# Initialize when center button is touched
@onevent 
def button_center():
    global beliefs, button_center, state
    if button_center == 0:
        nf_math_fill(beliefs, 1000 // 8)
        display_beliefs()
        state = 0

# Sense
def sense():
    global prox_ground_delta, THRESHOLD, hit, world, beliefs, P_HIT, P_MISS, sum_b
    
    # Check if on black (1) or not (0)
    if  prox_ground_delta[0] < THRESHOLD:
        if prox_ground_delta[1] < THRESHOLD :
            hit = 1
        else:
            hit = 0
    else:
        hit = 0


  # If hit matches world at each position
  #   multiply the beliefs by the hit/miss probabilities
    for i in range(8):
        if  hit == 1:
            if world[i] == 1:
                beliefs[i] = math_muldiv(beliefs[i], P_HIT, 10)
            else:
                beliefs[i] = math_muldiv(beliefs[i], P_MISS, 10)
        elif hit == 0:
            if world[i] == 0:
                beliefs[i] = math_muldiv(beliefs[i], P_HIT, 10)
            else:
                beliefs[i] = math_muldiv(beliefs[i], P_MISS, 10)

    # Compute the sum of the beliefs
    sum_b = 0
    for i in range(8):
        sum_b += beliefs[i]

    # Normalize
    for i in range(8):
        beliefs[i] = math_muldiv(beliefs[i], 1000, sum_b)

# Move right cyclic
def move():
    global beliefs
    
    temp = beliefs[7]
    #for i in 7:1 step -1 do
    i = 7
    while i > 0:
        beliefs[i] = beliefs[i-1]
        i = i - 1 
    beliefs[0] = temp

# Touch button forward to repeat sense -> move -> ...
@onevent 
def button_forward():
    global button_forward, state
    if button_forward == 0:
        if state == 0:
            sense()
            state = 1
        else: 
            move()
            state = 0
    display_beliefs()

<span style="color: #2980B9 ;">
    
> And now if you want to implement the continuous move of the robot:
    
You can vary a little the speed depending of the robot, so it will fit the rate of sensing.

In [ ]:
%%run_python

LED = 32            # scale factor for displaying beliefs in leds
THRESHOLD = 400     # value below black is detected
P_HIT = 9           # probability of detecting black when on black
                    #              or white when on white
P_MISS = 1          # probability of detecting black when on white
                    #              or white when on black
P_0 = 1
P_1 = 8
P_2 = 1
MOTOR = 100
TIMER = 1600
nb_round = 0

# The world: 2 black, 2 white, 3 black, 1 white
world   = [1, 1, 0, 0, 1, 1, 1, 0]

# Beliefs about the localisation
beliefs = [0, 0, 0, 0, 0, 0, 0, 0]

# Temporary array for computing motion with uncertainty
save    = [0, 0, 0, 0, 0, 0, 0, 0]

# state  0 = sense, 1 = move
# hit    1 = on black, 0 = on white
# temp   Variable for cyclic move
# sum_b  Sum of beliefs for normalization

# Display the beliefs in the circle leds
def display_beliefs():
    global beliefs, LED, leds_circle
    leds_circle = [beliefs[0]//LED, beliefs[1]//LED, beliefs[2]//LED, beliefs[3]//LED,
                   beliefs[4]//LED, beliefs[5]//LED, beliefs[6]//LED, beliefs[7]//LED]
        
# Initialize when center button is touched
@onevent 
def button_center():
    global beliefs, button_center, state, motor_right_target, motor_left_target, nb_round
    if button_center == 0:
        motor_right_target = 0
        motor_left_target  = 0
        nf_math_fill(beliefs, 1000 // 8)
        display_beliefs()
        state = 0
        nb_round = 0


# Touch button forward to start
@onevent
def button_forward():
    global button_forward, state, motor_right_target, motor_left_target, TIMER, MOTOR
    if button_forward == 0:
        if state == 0:
            state = 1
            display_beliefs()
            timer_period[0] = TIMER
            motor_left_target = MOTOR
            motor_right_target = MOTOR
        else:
            state = 0
            motor_left_target = 0
            motor_right_target = 0 

# Normalize
def normalize():
    global beliefs
    # Compute the sum of the beliefs
    sum_b = 0
    for i in range(8):
        sum_b += beliefs[i]

    # Normalize
    for i in range(8):
        beliefs[i] = math_muldiv(beliefs[i], 1000, sum_b)

# Sense
def sense():
    global prox_ground_delta, THRESHOLD, hit, world, beliefs, P_HIT, P_MISS, sum_b
    
    # Check if on black (1) or not (0)
    if  prox_ground_delta[0] < THRESHOLD:
        if prox_ground_delta[1] < THRESHOLD :
            hit = 1
        else:
            hit = 0
    else:
        hit = 0


  # If hit matches world at each position
  #   multiply the beliefs by the hit/miss probabilities
    for i in range(8):
        if  hit == 1:
            if world[i] == 1:
                beliefs[i] = math_muldiv(beliefs[i], P_HIT, 10)
            else:
                beliefs[i] = math_muldiv(beliefs[i], P_MISS, 10)
        elif hit == 0:
            if world[i] == 0:
                beliefs[i] = math_muldiv(beliefs[i], P_HIT, 10)
            else:
                beliefs[i] = math_muldiv(beliefs[i], P_MISS, 10)
                
            normalize()

# Move right cyclic with uncertainty 0, 1, 2 positions
def move():
    global beliefs, save, P_2, P_1, P_0
    # Save current beliefs and initialize beliefs to 0
    save = beliefs
    beliefs = [0, 0, 0, 0, 0, 0, 0, 0]

    # From positions 0,1,2, the robot can move to 2
    # ...
    # From positions 7,8=0,9=1, the robot can move to 1
    for i in range(2,10):
        beliefs[i % 8] += math_muldiv(save[(i - 2) % 8], P_2, 10)
        beliefs[i % 8] += math_muldiv(save[(i - 1) % 8], P_1, 10)
        beliefs[i % 8] += math_muldiv(save[i % 8], P_0, 10)

    normalize()

@onevent 
def timer0():
    global state, nb_round, motor_right_target, motor_left_target
    if state == 1:
        if nb_round < 8:
            sense()
            move()
            display_beliefs()
            nb_round += 1
        else:
            motor_right_target = 0
            motor_left_target  = 0
            
    #sound.freq(1000,10)